# 변수를 추출하고 학습데이터, Validation 데이터, Test데이터를 만들어보자

###  MariaDB에서 필요한 데이터를 조회하는 방법

In [18]:
DB_HOST = 'node02'
DB_USER = 'deepbio'
DB_PW = 'deepbio1234'
DB_NAME = 'deepbioDB'

In [19]:
import mysql.connector as mariadb
mariadb_connection = mariadb.connect(host=DB_HOST, user=DB_USER, password=DB_PW, database=DB_NAME)
cursor = mariadb_connection.cursor()
cursor.execute(
    "SELECT id ,disease, platform_type, center, platform, level, barcode, file_name " + 
    "FROM file_manifest " +
    "WHERE file_name like '%rsem.genes.normalized_results' limit 2")
for ID ,disease, platform_type, center, platform, level, barcode, file_name  in cursor:
    fullPath =  "{}/{}/{}__{}/{}/{}".format(disease, platform_type, center, platform, level, file_name) 
    print fullPath

#select count(*) from file_manifest where file_name like '%rsem.genes.normalized_results';
mariadb_connection.close()

THCA/RNASeqV2/UNC__IlluminaHiSeq_RNASeqV2/Level_3/unc.edu.63f7a81e-c404-49fc-bc72-52b2a81ea9a5.1366788.rsem.genes.normalized_results
SARC/RNASeqV2/UNC__IlluminaHiSeq_RNASeqV2/Level_3/unc.edu.349f670f-1447-4bb0-86ec-fba8734d50ff.2428615.rsem.genes.normalized_results


### HBASE에서 컨텐츠 데이터 조회하는 방법

In [24]:
import happybase
pool = happybase.ConnectionPool(size=3, host='node03')   

def getContents( pool, rowkey ) : 
    with pool.connection() as connection:
        table = connection.table('mRNA')
        print( table )
        #print( table.regions() )
        row = table.row(rowkey)
        return row['data:contents']   # prints the value of cf1:col1

In [27]:
print getContents( pool, 'unc.edu.63f7a81e-c404-49fc-bc72-52b2a81ea9a5.1366788.rsem.genes.normalized_results' )

<happybase.table.Table name='mRNA'>
gene_id	normalized_count
?|100130426	0.0000
?|100133144	1.1029
?|100134869	2.3618
?|10357	189.9697
?|10431	869.9293
?|136542	0.0000
?|155060	64.0970
?|26823	0.2887
?|280660	4.9083
?|317712	0.0000
?|340602	0.8662
?|388795	0.2887
?|390284	4.3309
?|391343	0.0000
?|391714	0.0000
?|404770	0.0000
?|441362	0.0000
?|442388	0.0000
?|553137	0.5775
?|57714	273.1341
?|645851	1.4436
?|652919	21.1260
?|653553	207.5935
?|728045	0.0000
?|728603	0.0000
?|728788	0.2887
?|729884	32.0485
?|8225	601.4148
?|90288	21.9431
A1BG|1	57.2773
A1CF|29974	0.0000
A2BP1|54715	297.6758
A2LD1|87769	93.1947
A2ML1|144568	0.0000
A2M|2	21763.5224
A4GALT|53947	339.2522
A4GNT|51146	0.2887
AAA1|404744	0.2887
AAAS|8086	533.8530
AACSL|729522	0.0000
AACS|65985	558.9721
AADACL2|344752	0.0000
AADACL3|126767	0.0000
AADACL4|343066	0.0000
AADAC|13	0.0000
AADAT|51166	40.1328
AAGAB|79719	1070.3046
AAK1|22848	1569.5106
AAMP|14	2729.3201
AANAT|15	0.5775
AARS2|57505	329.1439
AARSD1|80755	644.7235
AARS|16

In [28]:
print getContents( pool, 'unc.edu.349f670f-1447-4bb0-86ec-fba8734d50ff.2428615.rsem.genes.normalized_results' )

<happybase.table.Table name='mRNA'>
gene_id	normalized_count
?|100130426	0.0000
?|100133144	1.4054
?|100134869	6.0158
?|10357	168.7013
?|10431	1127.0872
?|136542	0.0000
?|155060	117.8108
?|26823	0.0000
?|280660	0.0000
?|317712	0.0000
?|340602	2.3191
?|388795	0.0000
?|390284	6.4935
?|391343	0.0000
?|391714	0.9276
?|404770	0.0000
?|441362	0.0000
?|442388	0.0000
?|553137	0.0000
?|57714	339.0538
?|645851	1.8553
?|652919	0.0000
?|653553	655.3803
?|728045	0.0000
?|728603	0.0000
?|728788	0.4638
?|729884	0.0000
?|8225	916.5121
?|90288	2.3191
A1BG|1	57.6020
A1CF|29974	0.0000
A2BP1|54715	2.3191
A2LD1|87769	73.1354
A2ML1|144568	59.3692
A2M|2	56118.6132
A4GALT|53947	128.0148
A4GNT|51146	0.0000
AAA1|404744	0.0000
AAAS|8086	730.9833
AACSL|729522	0.9276
AACS|65985	617.3469
AADACL2|344752	0.4638
AADACL3|126767	0.0000
AADACL4|343066	2.3191
AADAC|13	0.4638
AADAT|51166	137.7551
AAGAB|79719	1056.5863
AAK1|22848	1936.9202
AAMP|14	2938.7755
AANAT|15	0.0000
AARS2|57505	645.1531
AARSD1|80755	1408.1633
AARS|16

- gene_id 는 GENE NAME | GENE ID
- 예) A1CF|29974 
http://www.ncbi.nlm.nih.gov/gene/29974

### Disease별 데이터수 확인

In [6]:
import mysql.connector as mariadb
mariadb_connection = mariadb.connect(host=DB_HOST, user=DB_USER, password=DB_PW, database=DB_NAME)
cursor = mariadb_connection.cursor()
cursor.execute( 
    "SELECT disease, count(*) " +
    "FROM file_manifest " + 
    "WHERE file_name like '%rsem.genes.normalized_results'  " + 
    "GROUP BY disease ORDER BY disease" )
total_count = 0
for disease, count  in cursor:
    print "{} : {} ".format(disease, count)
    total_count += count

mariadb_connection.close() 

print "TotalCount :{}".format( total_count )

ACC : 79 
BLCA : 433 
BRCA : 1252 
CESC : 309 
CHOL : 45 
COAD : 547 
DLBC : 48 
ESCA : 196 
FPPP : 76 
GBM : 174 
HNSC : 566 
KICH : 91 
KIRC : 614 
KIRP : 323 
LAML : 173 
LGG : 534 
LIHC : 424 
LUAD : 600 
LUSC : 553 
MESO : 87 
OV : 309 
PAAD : 183 
PCPG : 187 
PRAD : 558 
READ : 177 
SARC : 265 
SKCM : 473 
STAD : 450 
TGCT : 156 
THCA : 572 
THYM : 122 
UCEC : 590 
UCS : 57 
UVM : 80 
TotalCount :11303


In [36]:
diseaseDic  = {'ACC'  : 1 
,'BLCA' : 2 
,'BRCA' : 3  
,'CESC' : 4 
,'CHOL' : 5 
,'COAD' : 6 
,'DLBC' : 7 
,'ESCA' : 8 
,'FPPP' : 9 
,'GBM'  : 10
,'HNSC' : 11
,'KICH' : 12
,'KIRC' : 13
,'KIRP' : 14
,'LAML' : 15
,'LGG'  : 16
,'LIHC' : 17
,'LUAD' : 18
,'LUSC' : 19
,'MESO' : 20
,'OV'   : 21
,'PAAD' : 22
,'PCPG' : 23
,'PRAD' : 24
,'READ' : 25
,'SARC' : 26
,'SKCM' : 27
,'STAD' : 28
,'TGCT' : 29
,'THCA' : 30
,'THYM' : 31
,'UCEC' : 32
,'UCS'  : 33
,'UVM'  : 34 }

In [67]:
def datasetFromDisease( disease, diseaseDic, debug=0 ) :

    import mysql.connector as mariadb
    mariadb_connection = mariadb.connect(host=DB_HOST, user=DB_USER, password=DB_PW, database=DB_NAME)
    cursor = mariadb_connection.cursor()
    cursor.execute(
        "SELECT COUNT(*)  FROM file_manifest " +
        "WHERE disease = '{}' AND file_name like '%rsem.genes.normalized_results' ".format(disease) )
    
    totalCount = cursor.fetchall()[0][0]
    train_set_cnt = totalCount * 70 /  100
    val_set_cnt   = totalCount * 15 /  100
    test_set_cnt  = totalCount * 15 /  100
    
    print "{}, TotalCount={}, TrainCount={}, ValCount={}, TestCount={}".format(disease, totalCount, train_set_cnt, val_set_cnt, test_set_cnt ) 
    
    train_set_cnt += (totalCount - train_set_cnt - val_set_cnt - test_set_cnt  )
    print "{}, TotalCount={}, TrainCount={}, ValCount={}, TestCount={}".format(disease, totalCount, train_set_cnt, val_set_cnt, test_set_cnt ) 
    
    cursor.execute( 
        "SELECT disease, file_name " +
        "FROM file_manifest " +
        "WHERE disease = '{}' AND file_name like '%rsem.genes.normalized_results' " +
        "ORDER BY rand() ".format(disease)  )
    
    train_set = []
    val_set = []
    test_set = []
    
    cnt = 0;
    for disease, file_name  in cursor:
        #print "{} : {} ".format(disease, file_name)
        if  train_set_cnt > cnt :
            train_set.append( (file_name, diseaseDic[ disease  ])  )
        elif  val_set_cnt  + train_set_cnt > cnt : 
            val_set.append( (file_name, diseaseDic[ disease  ])  )
        else :
            test_set.append( (file_name, diseaseDic[ disease  ])  )
        cnt += 1
    
    if 1 == debug :
        for file_name, y in train_set : print "train : {},  {} ".format(file_name, y)
        for file_name, y in val_set : print "val : {},  {} ".format(file_name, y)
        for file_name, y in test_set : print "test : {},  {} ".format(file_name, y)
      
    mariadb_connection.close() 
    return [train_set, val_set, test_set]

In [68]:
DATA =  datasetFromDisease( 'BLCA', diseaseDic )

BLCA, TotalCount=433, TrainCount=303, ValCount=64, TestCount=64
BLCA, TotalCount=433, TrainCount=305, ValCount=64, TestCount=64


In [69]:
DATA = datasetFromDisease( 'ACC', diseaseDic )

ACC, TotalCount=79, TrainCount=55, ValCount=11, TestCount=11
ACC, TotalCount=79, TrainCount=57, ValCount=11, TestCount=11


In [70]:
diseases = diseaseDic.keys() 
for disease in diseases : 
    datasetFromDisease( disease, diseaseDic )

ACC, TotalCount=79, TrainCount=55, ValCount=11, TestCount=11
ACC, TotalCount=79, TrainCount=57, ValCount=11, TestCount=11
DLBC, TotalCount=48, TrainCount=33, ValCount=7, TestCount=7
DLBC, TotalCount=48, TrainCount=34, ValCount=7, TestCount=7
PAAD, TotalCount=183, TrainCount=128, ValCount=27, TestCount=27
PAAD, TotalCount=183, TrainCount=129, ValCount=27, TestCount=27
TGCT, TotalCount=156, TrainCount=109, ValCount=23, TestCount=23
TGCT, TotalCount=156, TrainCount=110, ValCount=23, TestCount=23
LUSC, TotalCount=553, TrainCount=387, ValCount=82, TestCount=82
LUSC, TotalCount=553, TrainCount=389, ValCount=82, TestCount=82
READ, TotalCount=177, TrainCount=123, ValCount=26, TestCount=26
READ, TotalCount=177, TrainCount=125, ValCount=26, TestCount=26
GBM, TotalCount=174, TrainCount=121, ValCount=26, TestCount=26
GBM, TotalCount=174, TrainCount=122, ValCount=26, TestCount=26
THCA, TotalCount=572, TrainCount=400, ValCount=85, TestCount=85
THCA, TotalCount=572, TrainCount=402, ValCount=85, TestC

http://stackoverflow.com/questions/26107927/how-to-put-my-dataset-in-a-pkl-file-in-the-exact-format-and-data-structure-used

```
from PIL import Image
from numpy import genfromtxt
import gzip, cPickle
from glob import glob
import numpy as np
import pandas as pd
Data, y = dir_to_dataset("trainMNISTForm\\*.BMP","trainLabels.csv")
# Data and labels are read 

train_set_x = Data[:2093]
val_set_x = Data[2094:4187]
test_set_x = Data[4188:6281]
train_set_y = y[:2093]
val_set_y = y[2094:4187]
test_set_y = y[4188:6281]
# Divided dataset into 3 parts. I had 6281 images.

train_set = train_set_x, train_set_y
val_set = val_set_x, val_set_y
test_set = test_set_x, val_set_y

dataset = [train_set, val_set, test_set]

f = gzip.open('file.pkl.gz','wb')
cPickle.dump(dataset, f, protocol=2)
f.close()

```